In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import os
import geopandas as gpd

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel,
)

# Porcentagem do distrito com cobertura vegetal

In [3]:
data_path = get_data_diretorio()

# Dependências

Este notebook é dependente dos assets resultante dos notebooks 'overlay_distrito_cobertura_vegetal' e '../../arborizacao_viaria/carregar_dados/malha_distritos'

In [4]:
overlay_path = os.path.join(
    data_path, 
    'assets',
    'cobertura_vegetal',
    'overlay_cobertura_vegetal_distritos.parquet'
)
overlay_gdf = gpd.read_parquet(overlay_path)

In [5]:
overlay_gdf.sample(2)

,cd_cob_veg,cd_identi0,cd_categor,cd_subcate,cd_subcat0,qt_altura_,area_cob_veg,CD_MUN,NM_MUN,CD_DIST,...,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_recort,perc_recort
205815,204013.0,204566.0,13.0,0.0,0.0,0.0,139.368005,3550308,São Paulo,355030837,...,350001,São Paulo,3550308,São Paulo/SP,14.699172,210960,85467,"POLYGON ((350518.811 7397160.072, 350519.367 7...",139.368005,1.0
240792,238829.0,239939.0,11.0,0.0,0.0,0.0,98.716906,3550308,São Paulo,355030820,...,350001,São Paulo,3550308,São Paulo/SP,7.785398,84397,37987,"POLYGON ((342003.367 7395421.249, 342006.979 7...",98.716906,1.0


In [6]:
distritos_path = os.path.join(data_path, 'assets', 'distrito_ibge.parquet')
gdf_distritos = gpd.read_parquet(distritos_path)

# Merge dos gdfs

In [7]:
gdf_dist_cobveg = gdf_distritos.merge(
    overlay_gdf[[
        'cd_cob_veg',
        'area_recort',
        'perc_recort',
        'CD_DIST'
    ]],
    on='CD_DIST'
)

In [8]:
gdf_dist_cobveg.shape

(274601, 17)

In [9]:
gdf_distritos.shape

(96, 14)

# Calcular área de cobertura vegetal por distrito

In [10]:
for distrito in gdf_distritos['CD_DIST']:
    area_cobveg = sum(
            gdf_dist_cobveg['area_recort']
            .loc[gdf_dist_cobveg['CD_DIST']==distrito]
    )
    
    (gdf_distritos
        .loc[
            gdf_distritos['CD_DIST']==distrito, 
            'area_dist_cobveg'
        ])= area_cobveg

In [11]:
gdf_distritos.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg
60,3550308,São Paulo,355030861,PERUS,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,23.636765,87716,34379,"POLYGON ((321923.417 7412300.951, 321941.7 741...",1.571995e+07
16,3550308,São Paulo,355030817,CAMPO LIMPO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.579836,236162,97934,"POLYGON ((322009.936 7384634.641, 322009.855 7...",2.605076e+06


# Cálculo da porcentagem da área

Antes de tudo, vou confirmar se a coluna 'AREA_KM_2' pode ser usada como base para o cálculo

In [12]:
x=(
    gdf_distritos['geometry']
    .area
    .loc[gdf_distritos['CD_DIST']=='355030891']
)
print(x.round(3))

90    8896709.007
dtype: float64


In [13]:
print(
    gdf_distritos['AREA_KM2']
    .loc[gdf_distritos['CD_DIST']=='355030891']
)

90    8.898548
Name: AREA_KM2, dtype: float64


Aparentemente a área proveniente do cálculo da geometria não é em Km², então faremos uma nova coluna para calcular a área do distrito segundo a geometria

In [14]:
gdf_distritos['area_distrito'] = gdf_distritos['geometry'].area


In [15]:
gdf_distritos.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito
95,3550308,São Paulo,355030896,LAJEADO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,8.902602,164391,63066,"POLYGON ((356021.63 7395589.489, 356021.497 73...",1.225673e+06,8.900028e+06
5,3550308,São Paulo,355030806,BARRA FUNDA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,5.884671,33436,16962,"POLYGON ((329146.056 7396609.887, 329100.802 7...",1.128738e+06,5.884207e+06


In [16]:
gdf_distritos['pct_dist_cobveg'] = (
    gdf_distritos['area_dist_cobveg'] / gdf_distritos['area_distrito']
)

In [17]:
gdf_distritos.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito,pct_dist_cobveg
59,3550308,São Paulo,355030860,PERDIZES,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,6.315923,102391,54775,"POLYGON ((328317.572 7394626.129, 328266.485 7...",1.369631e+06,6.315464e+06,0.216869
21,3550308,São Paulo,355030822,CIDADE ADEMAR,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.233190,249218,103713,"POLYGON ((333467.739 7379561.767, 333458.192 7...",1.238026e+06,1.223203e+07,0.101212


# Conferir compatibilidade e consistência

Agora vamos procurar por alguma inconsistência no cálculo da porcentagem. Caso algum valor de 'pct_dist_cobveg' seja maior ou igual a 1, seria como se a totalidade da área do distrito, ou mais do que ela, é de cobertura vegetal

In [18]:
gdf_distritos[gdf_distritos['pct_dist_cobveg']>=1]

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito,pct_dist_cobveg


In [19]:
sum(gdf_distritos['pct_dist_cobveg'].isnull())

0

In [20]:
max_pct=gdf_distritos['pct_dist_cobveg'].max()
max_name=gdf_distritos['NM_DIST'].loc[gdf_distritos['pct_dist_cobveg']==max_pct].iloc[0]
print(f'{max_name} é o distrito com a maior porcentagem de cobertura vegetal: {round(max_pct*100, 2)}%')

MARSILAC é o distrito com a maior porcentagem de cobertura vegetal: 97.12%


In [21]:
min_pct=gdf_distritos['pct_dist_cobveg'].min()
min_name = gdf_distritos['NM_DIST'].loc[gdf_distritos['pct_dist_cobveg']==min_pct].iloc[0]
print(f'{min_name} é o distrito com a menor porcentagem de cobertura vegetal: {round(min_pct*100, 2)}%')

BRAS é o distrito com a menor porcentagem de cobertura vegetal: 5.57%


# Porcentagem do Município de São Paulo

In [22]:
gdf_distritos.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito,pct_dist_cobveg
50,3550308,São Paulo,355030851,MANDAQUI,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,13.202709,103665,45143,"POLYGON ((332902.165 7405221.112, 332904.48 74...",7.460771e+06,1.320131e+07,0.565154
29,3550308,São Paulo,355030830,GRAJAU,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,92.578049,384873,154205,"POLYGON ((327239.195 7362453.213, 327195.452 7...",5.057260e+07,9.257010e+07,0.546317


In [23]:
area_sp = sum(gdf_distritos['area_distrito'])
area_cobveg_total = sum(gdf_distritos['area_dist_cobveg'])
pct_sp = area_cobveg_total / area_sp
print(f"{round(pct_sp*100, 2)}% do município de São Paulo é área de cobertura vegetal")

47.96% do município de São Paulo é área de cobertura vegetal


# Salvar gdf

In [24]:
save_parquet_excel(
    gdf_distritos, 
    fname= 'distritos_cobertura_vegetal',
    data_path = data_path,
    data_subpath = os.path.join(
        'assets',
        'cobertura_vegetal'
    )
)